In [1]:
using IJulia
installkernel("julia_ITensors","--sysimage=~/.julia/sysimages/sys_itensors.so")

[ Info: Installing julia_ITensors kernelspec in /Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9


"/Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9"

In [2]:
using ITensors
using ITensors.HDF5
using DataFrames

# Lattice, DMRG function

In [3]:
function spinchain_open(L)
    
    interactions = DataFrame(["$spin"=>[] for spin in 1:L-1])
    
    for spin in 1:L-1
        neighbor = spin+1
        push!(interactions[!,"$spin"],[spin,neighbor])
    end

    return interactions
end

spinchain_open (generic function with 1 method)

In [4]:
function dmrg_1D_ising(L::Int, J, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
        os += 4*J,"Sz",i,"Sz",j
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1);
    energy_per_spin = energy / L
    
    energy_est = inner(psi',H,psi)
    energy_per_spin_est = energy_est/L
    
    println("\nA sample from the optimized MPS looks like:\n",sample(psi))
    
    if save   
        f = h5open("../out/ising_fm_1d_L$(L)/sanity_check/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
    
    for i =1:L
        println("M",i)
        println(psi[i])
    end
    
    return energy_per_spin, energy_per_spin_est, psi
end

dmrg_1D_ising (generic function with 1 method)

In [5]:
function dmrg_1D_tfim(L::Int, J, h, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
        os += 4*J,"Sz",i,"Sz",j
    end
    for k=1:L
        os += 2*h,"Sx",k
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setmindim!(sweeps,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
    
    energy_est = inner(psi',H,psi)
    energy_per_spin_est = energy_est/L
    
    if save
        f = h5open("../out/ising_fm_1d_L$(L)_h1/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
    
#     println("")
#     @show psi
    
    return energy_per_spin,energy_per_spin_est,psi
end

dmrg_1D_tfim (generic function with 1 method)

In [6]:
function dmrg_1D_heis(L::Int, J, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
    os += 4*J,"Sz",j,"Sz",k
    os += 4*J/2,"S+",j,"S-",k
    os += 4*J/2,"S-",j,"S+",k
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setmindim!(sweeps, 10,20,100,100,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
        
    if save   
        f = h5open("../out/heis_afm_1d_L$(L)/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
        
    return energy_per_spin, energy_per_spin, psi
end

dmrg_1D_heis (generic function with 1 method)

# Try growing algo

In [7]:
function grow(smaller_psi,larger_psi)
    
    psi_s = deepcopy(smaller_psi)
    psi_l = deepcopy(larger_psi)
    N_s = length(psi_s)
    N_l = length(psi_l)
    
    # Step 1: Get MPS in center canonical form with middle lambda
    half_s = Int(N_s/2)
    half_l = Int(N_l/2)
    psi_s = orthogonalize(psi_s,half_s)
    psi_l = orthogonalize(psi_l,half_l)
    
    combiner_s = combiner(siteind(psi_s,half_s),linkind(psi_s,half_s-1); tags="site, left link")
    combined_s = combinedind(combiner_s)
    U_s,lambda_s,V_s = svd(psi_s[half_s]*combiner_s,combined_s,lefttags = "alpha$half_s",righttags = "beta$half_s");
    U_s = dag(combiner_s) * U_s;
    center_alpha_s,center_beta_s = inds(lambda_s);
    inv_lambda_s = ITensor(inv(Matrix(lambda_s,center_alpha_s,center_beta_s)),center_beta_s,center_alpha_s);

    combiner_ = combiner(siteind(psi_l,half_l),linkind(psi_l,half_l-1); tags="site, left link")
    combined_ = combinedind(combiner_)
    U_l,lambda_l,V_l = svd(psi_l[half_l]*combiner_,combined_,lefttags = "alpha$half_l",righttags = "beta$half_l")
    U_l = dag(combiner_) * U_l;

    # Re-tag and Re-name matrices
    link_left = commonind(psi_l[half_l - 1],U_l)
    center_alpha = Index(dim(link_left),"alpha$half_s")
    link_right = commonind(V_l*psi_l[half_l + 1],psi_l[half_l + 2])
    center_beta = Index(dim(link_right),"beta$half_s")
            
    replaceind!(inv_lambda_s,center_beta_s,center_beta)
    replaceind!(inv_lambda_s,center_alpha_s,center_alpha)

    AR2 = replaceind!(psi_l[half_l - 1],link_left,center_alpha)
    AR1 = replaceind!(U_l,link_left,center_alpha)
    BL1 = replaceind!(V_l*psi_l[half_l + 1],link_right,center_beta)
    BL2 = replaceind!(psi_l[half_l + 2],link_right,center_beta)
    
    sitepR = prime(inds(BL1,"Site")[1])
    BR = replaceind(BL1,inds(BL1,"Site")[1],sitepR)
    sitepL = prime(inds(AR1,"Site")[1])
    AL = replaceind(AR1,inds(AR1,"Site")[1],sitepL)
    
    # Stitch together the new MPS
    new_MPS = MPS(N_l + 2)
    N_new = length(new_MPS)
    
    for i in 1:(half_l-2)
        new_MPS[i] = psi_l[i]
    end
    
    rightlink = inds(AR2,"alpha$(half_l-1)")[1]
    newrightlink = Index(dim(rightlink),"Link,l=$(half_l-1)")
    new_MPS[half_l-1] = replaceind(AR2,rightlink,newrightlink)
    
    leftlink = rightlink
    newleftlink = newrightlink
    rightlink = inds(AR1*lambda_l,"beta$(half_l)")[1]
    newrightlink = Index(dim(rightlink),"Link,l=$(half_l)")
    new_MPS[half_l] = replaceinds(AR1*lambda_l,(leftlink,rightlink),(newleftlink,newrightlink))
    
    leftlink = rightlink
    newleftlink = newrightlink
    rightlink = inds(BR*inv_lambda_s,"alpha$(half_l-1)")[1]
    newrightlink = Index(dim(rightlink),"Link,l=$(half_l+1)")
    site = inds(noprime(BR*inv_lambda_s),"Site")[1]
    newsiteind = Index(dim(site),"S=1/2,Site,n=$(half_l+1)")
    new_MPS[half_l+1] = replaceinds(noprime(BR*inv_lambda_s),(leftlink,rightlink,site),(newleftlink,newrightlink,newsiteind))

    leftlink = rightlink
    newleftlink = newrightlink
    rightlink = inds(AL*lambda_l,"beta$(half_l)")[1]
    newrightlink = Index(dim(rightlink),"Link,l=$(half_l+2)")
    site = inds(noprime(AL*lambda_l),"Site")[1]
    newsiteind = Index(dim(site),"S=1/2,Site,n=$(half_l+2)")
    new_MPS[half_l+2] = replaceinds(noprime(AL*lambda_l),(leftlink,rightlink,site),(newleftlink,newrightlink,newsiteind))

    leftlink = rightlink
    newleftlink = newrightlink
    rightlink = inds(BL1,"beta$(half_l-1)")[1]
    newrightlink = Index(dim(rightlink),"Link,l=$(half_l+3)")
    site = inds(BL1,"Site")[1]
    newsiteind = Index(dim(site),"S=1/2,Site,n=$(half_l+3)")
    new_MPS[half_l+3] = replaceinds(BL1,(leftlink,rightlink,site),(newleftlink,newrightlink,newsiteind))

    # need to add a flag for if this is the last site ...
    leftlink = rightlink
    newleftlink = newrightlink
    rightlink = inds(BL2,"Link,l=$(half_l+2)")[1]
    newrightlink = Index(dim(rightlink),"Link,l=$(half_l+4)")
    site = inds(BL2,"Site")[1]
    newsiteind = Index(dim(site),"S=1/2,Site,n=$(half_l+4)")
    new_MPS[half_l+4] = replaceinds(BL2,(leftlink,rightlink,site),(newleftlink,newrightlink,newsiteind))
    
#     println(norm(new_MPS[half_l-1]*new_MPS[half_l]*new_MPS[half_l+1]*new_MPS[half_l+2]*new_MPS[half_l+3]*new_MPS[half_l+4]))
#     println(norm(AR2*AR1*lambda_l*BR*inv_lambda_s*AL*lambda_l*BL1*BL2))
    
    for i in (half_l+3):N_l
        siteind = inds(psi_l[i],"Site")[1]
        newsiteind = Index(dim(siteind),"S=1/2,Site,n=$(i+2)")
        leftlink = rightlink
        newleftlink = newrightlink
        new_MPS[i+2] = replaceinds(psi_l[i],(siteind,leftlink),(newsiteind,newleftlink))
        if i < N_l
            rightlink = inds(psi_l[i],"l=$(i)")[1]
            newrightlink = Index(dim(rightlink),"Link,l=$(i+2)")
            new_MPS[i+2] = replaceind!(new_MPS[i+2],rightlink,newrightlink)
        end
    end
        
    return new_MPS
end

grow (generic function with 1 method)

In [18]:
function grow_MPS(L_original,L_final,save::Bool)
    
    n_grows = Int((L_final-L_original)/2)    
#     _,_,psi0 = dmrg_1D_heis((L_original-2),1.,5,"open",false);
#     _,_,psi1 = dmrg_1D_heis(L_original,1.,5,"open",true);
    _,_,psi0 = dmrg_1D_tfim((L_original-2),-1.,-1.,2,"open",false);
    _,_,psi1 = dmrg_1D_tfim(L_original,-1.,-1.,2,"open",true);
    
    for n in 1:n_grows
        psi_new = grow(psi0,psi1)
        psi0 = psi1
        psi1 = psi_new
    end
    
    if save
        f = h5open("../out/ising_fm_1d_L$(L_original)_h1/init_$(L_final).hdf5","w") 
#         f = h5open("../out/heis_afm_1d_L$(L_original)/init_$(L_final).hdf5","w") 
        write(f,"psi",psi1)
        close(f)
    end
    
    return psi1
end

grow_MPS (generic function with 1 method)

In [21]:
psi = grow_MPS(10,20,true)


Running DMRG for a length 8 spin chain

After sweep 1 energy=-9.734486113385838  maxlinkdim=10 maxerr=1.71E-04 time=0.014
After sweep 2 energy=-9.837621425487693  maxlinkdim=15 maxerr=4.94E-11 time=0.013
After sweep 3 energy=-9.837951420486217  maxlinkdim=7 maxerr=7.66E-11 time=0.013
After sweep 4 energy=-9.837951446196266  maxlinkdim=6 maxerr=7.06E-11 time=0.029
After sweep 5 energy=-9.832706741056095  maxlinkdim=2 maxerr=3.10E-04 time=0.008
After sweep 6 energy=-9.832729175527705  maxlinkdim=2 maxerr=2.29E-04 time=0.008
After sweep 7 energy=-9.832729468327724  maxlinkdim=2 maxerr=2.29E-04 time=0.007
After sweep 8 energy=-9.832729472751277  maxlinkdim=2 maxerr=2.29E-04 time=0.009
After sweep 9 energy=-9.832729472819505  maxlinkdim=2 maxerr=2.29E-04 time=0.007
After sweep 10 energy=-9.832729472820567  maxlinkdim=2 maxerr=2.29E-04 time=0.010

Running DMRG for a length 10 spin chain

After sweep 1 energy=-12.360654630285262  maxlinkdim=10 maxerr=3.84E-04 time=0.025
After sweep 2 energy=

MPS
[1] ((dim=2|id=647|"S=1/2,Site,n=1"), (dim=2|id=778|"Link,l=1"))
[2] ((dim=2|id=400|"S=1/2,Site,n=2"), (dim=2|id=778|"Link,l=1"), (dim=2|id=387|"Link,l=2"))
[3] ((dim=2|id=578|"S=1/2,Site,n=3"), (dim=2|id=387|"Link,l=2"), (dim=2|id=997|"Link,l=3"))
[4] ((dim=2|id=519|"S=1/2,Site,n=4"), (dim=2|id=997|"Link,l=3"), (dim=2|id=508|"Link,l=4"))
[5] ((dim=2|id=950|"S=1/2,Site,n=5"), (dim=2|id=508|"Link,l=4"), (dim=2|id=767|"Link,l=5"))
[6] ((dim=2|id=324|"S=1/2,Site,n=6"), (dim=2|id=767|"Link,l=5"), (dim=2|id=942|"Link,l=6"))
[7] ((dim=2|id=975|"S=1/2,Site,n=7"), (dim=2|id=942|"Link,l=6"), (dim=2|id=386|"Link,l=7"))
[8] ((dim=2|id=873|"S=1/2,Site,n=8"), (dim=2|id=386|"Link,l=7"), (dim=2|id=61|"Link,l=8"))
[9] ((dim=2|id=198|"S=1/2,Site,n=9"), (dim=2|id=61|"Link,l=8"), (dim=2|id=579|"Link,l=9"))
[10] ((dim=2|id=579|"Link,l=9"), (dim=2|id=806|"S=1/2,Site,n=10"), (dim=2|id=638|"Link,l=10"))
[11] ((dim=2|id=781|"S=1/2,Site,n=11"), (dim=2|id=638|"Link,l=10"), (dim=2|id=493|"Link,l=11"))
[12] (

MPS
[1] ((dim=2|id=647|"S=1/2,Site,n=1"), (dim=1|id=743|"Link,l=1"))
[2] ((dim=1|id=743|"Link,l=1"), (dim=2|id=400|"S=1/2,Site,n=2"), (dim=1|id=326|"Link,l=2"))
[3] ((dim=1|id=326|"Link,l=2"), (dim=2|id=578|"S=1/2,Site,n=3"), (dim=1|id=914|"Link,l=3"))
[4] ((dim=1|id=914|"Link,l=3"), (dim=2|id=519|"S=1/2,Site,n=4"), (dim=1|id=719|"Link,l=4"))
[5] ((dim=1|id=719|"Link,l=4"), (dim=2|id=950|"S=1/2,Site,n=5"), (dim=1|id=514|"Link,l=5"))
[6] ((dim=1|id=514|"Link,l=5"), (dim=2|id=324|"S=1/2,Site,n=6"), (dim=1|id=904|"Link,l=6"))
[7] ((dim=1|id=904|"Link,l=6"), (dim=2|id=975|"S=1/2,Site,n=7"), (dim=1|id=337|"Link,l=7"))
[8] ((dim=1|id=337|"Link,l=7"), (dim=2|id=873|"S=1/2,Site,n=8"), (dim=1|id=115|"Link,l=8"))
[9] ((dim=1|id=115|"Link,l=8"), (dim=2|id=198|"S=1/2,Site,n=9"), (dim=1|id=527|"Link,l=9"))
[10] ((dim=1|id=527|"Link,l=9"), (dim=2|id=806|"S=1/2,Site,n=10"), (dim=1|id=73|"Link,l=10"))
[11] ((dim=1|id=73|"Link,l=10"), (dim=2|id=781|"S=1/2,Site,n=11"), (dim=1|id=182|"Link,l=11"))
[12] (

# Test Grow Algo

In [ ]:
_,_,psi0 = dmrg_1D_tfim(8,-1.,-1.,2,"open",false);
psi1 = deepcopy(psi10);

In [ ]:
for n in 1:5
    println(n)
    psi_new = grow(psi0,psi1)
    psi0 = psi1
    psi1 = psi_new
    @show inds(psi1)
end

In [ ]:
@show psi1

# Step 1: Obtain wavefunction for a 2-site lattice and a 4-site lattice. 

In [ ]:
_,_,psi0 = dmrg_1D_ising(2,-1,2,"open",false)
_,_,psi1 = dmrg_1D_ising(4,-1,2,"open",false);

println("")
println("INDICES")
println(inds(psi0))
println(inds(psi1))

These MPS are returned in left-canonical form by the DMRG function, but we need them to be in center canonical form.

In [ ]:
N0 = 2
half0 = Int(N0/2)
psi0 = orthogonalize!(psi0,half0)
U0,lambda0,V0 = svd(psi0[half0],siteind(psi0,half0),lefttags = "alpha1",righttags = "beta1");


println("----New MPS----")
println(inds(U0))
println(inds(lambda0))
println(inds(V0*psi0[2]))

In [ ]:
N1 = 4
half1 = Int(N1/2)
psi1 = orthogonalize!(psi1,half1)
combiner_ = combiner(siteind(psi1,half1),linkind(psi1,half1-1); tags="site, left link")
combined_ = combinedind(combiner_)
U1,lambda1,V1 = svd(psi1[half1]*combiner_,combined_,lefttags = "alpha2",righttags = "beta2")
U1 = dag(combiner_) * U1;

println("----New MPS----")
println(inds(psi1[1]))
println(inds(U1))
println(inds(lambda1))
println(inds(V1*psi1[3]))
println(inds(psi1[4]))

Now these matrices are in the correct form:
- $\vert \Psi_0\rangle = A_0 \Lambda_0 B_0$
- $\vert \Psi_1\rangle = A_0 A_1 \Lambda_1 B_1 B_0$

# Ian's Method

Step 2 & 3: Rotate the center matrix of $\vert \Psi_1\rangle$ one step to the left and rotate the same center matrix one step to the right.

Step 2 (rotate to the left):

In [ ]:
j_ = 1
psi1L = orthogonalize(psi1,j_)
UL,lambdaL,VL = svd(psi1L[j_],siteind(psi1L,j_),lefttags = "alpha1",righttags = "alpha2")
M1 = UL
B2 = VL*psi1L[2]
B1 = psi1L[3]
B0 = psi1L[4]
println("----New----")
println(inds(M1))
println(inds(lambdaL))
println(inds(B2))
println(inds(B1))
println(inds(B0))
println("---Check full contraction---")
println(M1*lambdaL*B2*B1*B0)

Step 3 (rotate to the right):

In [ ]:
j_ = 4
psi1R = orthogonalize(psi1,j_)
UR,lambdaR,VR = svd(psi1R[j_],linkind(psi1R,j_-1),lefttags="beta2",righttags="beta1")
A0 = psi1R[1]
A1 = psi1R[2]
A2 = psi1R[3]*UR
M4 = VR
println("----New----")
println(inds(A0))
println(inds(A1))
println(inds(A2))
println(inds(lambdaR))
println(inds(M4))
println("---Check full contraction---")
println(A0*A1*A2*lambdaR*M4)

Step 4: Stitch together new MPS which has been grown by two sites.

In [ ]:
# println(inds(lambdaL,"alpha1")[1])
# println(inds(lambdaR,"beta1")[1])

alpha1_0,beta1_0 = inds(lambda0);
inv_lambda0 = ITensor(inv(Matrix(lambda0,alpha1_0,beta1_0)),beta1_0,alpha1_0);

println(inds(inv_lambda0))
inv_lambda0 = replaceind!(inv_lambda0,inds(lambda0,"beta1")[1],inds(lambdaR,"beta1")[1])
println(inds(inv_lambda0))
inv_lambda0 = replaceind!(inv_lambda0,inds(lambda0,"alpha1")[1],inds(lambdaL,"alpha1")[1])
println(inds(inv_lambda0))

In [ ]:
println(inds(B2,"Site")[1])
site2p = prime(inds(B2,"Site")[1])
B2R = replaceind(B2,inds(B2,"Site")[1],site2p)
println(B2R)
println(B2)

In [ ]:
println(inds(B1,"Site")[1])
site3p = prime(inds(B1,"Site")[1])
B1R = replaceind(B1,inds(B1,"Site")[1],site3p)
println(B1R)
println(B1)

In [ ]:
println("---Check full contraction---")
Ian_T = A0*A1*A2*lambdaR*inv_lambda0*lambdaL*B2R*B1R*B0;
println(norm(Ian_T))
println(Ian_T)

# Miles' method

In [ ]:
link1 = commonind(psi1[1],U1)
alpha1 = Index(dim(link1),"alpha1")
link3 = commonind(V1*psi1[3],psi1[4])
beta1 = Index(dim(link3),"beta1")

println(beta1,alpha1)

In [ ]:
alpha1_0,beta1_0 = inds(lambda0);
inv_lambda0 = ITensor(inv(Matrix(lambda0,alpha1_0,beta1_0)),beta1_0,alpha1_0);

println(inds(inv_lambda0))
inv_lambda0 = replaceind!(inv_lambda0,beta1_0,beta1)
println(inds(inv_lambda0))
inv_lambda0 = replaceind!(inv_lambda0,alpha1_0,alpha1)
println(inds(inv_lambda0))

In [ ]:
A1 = replaceind!(psi1[1],link1,alpha1)
A2 = replaceind!(U1,link1,alpha1)
B2 = replaceind!(V1*psi1[3],link3,beta1)
B1 = replaceind!(psi1[4],link3,beta1)

println("----New----")
println(inds(A1))
println(inds(A2))
println(inds(lambda1))
println(inds(B2))
println(inds(B1))
# println("----Old-----")
# println(psi1[1],psi1[2],psi1[3],psi1[4])
# println("----Compare Old & New----")
# println(psi1[1]*U1*lambda1*V1*psi1[3]*psi1[4])
# println(psi1[1]*psi1[2]*psi1[3]*psi1[4])

In [ ]:
site3p = prime(inds(B2,"Site")[1])
B2R = replaceind(B2,inds(B2,"Site")[1],site3p)
println(B2R)
println(B2)

In [ ]:
# println(inds(A2,"Site")[1])
site2p = prime(inds(A2,"Site")[1])
A2R = replaceind(A2,inds(A2,"Site")[1],site2p)
println(A2R)
println(A2)

In [ ]:
println(inds(A1))
println(inds(A2))
println(inds(lambda1))
println(inds(B2))
println(inds(inv_lambda0))
println(inds(A2))
println(inds(lambda1))
println(inds(B2))
println(inds(B1))

In [ ]:
println("---Check full contraction---")
Miles_T = A1*A2*lambda1*B2R*inv_lambda0*A2R*lambda1*B2*B1
println(norm(Miles_T))
println(Miles_T)